# STA 141B: Homework 4
Winter 2018

## Student Information

After the colons (in the same line) please write just your first name, last name, and the 9 digit student ID number below.

First Name: Mitchell

Last Name: Layton

Student ID: 912307956

## Instructions

We use a script that extracts your answers by looking for cells in between the cells containing the exercise statements.  So you 

- MUST add cells in between the exercise statements and add answers within them and
- MUST NOT modify the existing cells, particularly not the problem statement

To make markdown, please switch the cell type to markdown (from code) - you can hit 'm' when you are in command mode - and use the markdown language.  For a brief tutorial see: https://daringfireball.net/projects/markdown/syntax


## Requesting freshness

In this portion, you'll combine the assignment 3 data set with nutrition data from the [USDA Food Composition Databases](https://ndb.nal.usda.gov/ndb/search/list). The CSV file `fresh.csv` contains the fresh fruits and vegetables data you extracted in assignment 3.

The USDA Food Composition Databases have a [documented](https://ndb.nal.usda.gov/ndb/doc/index) web API that returns data in JSON format . You need a key in order to use the API. Only 1000 requests are allowed per hour, so it would be a good idea to use [caching][requests_cache].

[Sign up for an API key here](https://api.data.gov/signup/). The key will work with any Data.gov API. You may need the key again later in the quarter, so make sure you save it.

These modules may be useful:

* [requests](http://docs.python-requests.org/en/master/user/quickstart/)
* [requests_cache][]
* [urlparse](https://docs.python.org/2/library/urlparse.html)
* [pandas](http://pandas.pydata.org/pandas-docs/stable/)

[requests_cache]: https://pypi.python.org/pypi/requests-cache

__Exercise 1.1__ (20 pts)

Read the [search request documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-SEARCH.md), then write a function called `ndb_search()` that makes a search request. The function should accept the search term as an argument. The function should return the search result items as a list (for 0 items, return an empty list).

Note that the search url is: `https://api.nal.usda.gov/ndb/search`

As an example, a search for `"quail eggs"` should return this list:

```python
[{u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'CHAOKOH, QUAIL EGG IN BRINE, UPC: 044738074186',
  u'ndbno': u'45094707',
  u'offset': 0},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'L&W, QUAIL EGGS, UPC: 024072000256',
  u'ndbno': u'45094890',
  u'offset': 1},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'BUDDHA, QUAIL EGGS IN BRINE, UPC: 761934535098',
  u'ndbno': u'45099560',
  u'offset': 2},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'GRAN SABANA, QUAIL EGGS, UPC: 819140010103',
  u'ndbno': u'45169279',
  u'offset': 3},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u"D'ARTAGNAN, QUAIL EGGS, UPC: 736622102630",
  u'ndbno': u'45178254',
  u'offset': 4},
 {u'ds': u'SR',
  u'group': u'Dairy and Egg Products',
  u'name': u'Egg, quail, whole, fresh, raw',
  u'ndbno': u'01140',
  u'offset': 5}]
```

As usual, make sure you document and test your function.

In [1]:
import requests

my_api_key = 'qGHsT1sXfETbsOnmXQGHuqFwtucab2rrMntX2kb4'

def ndb_search(search_term, key):
    """
    Input 1: Search term for "q" parameter in API search
    Input 2: Owned API key for access
    
    Output: List of the "item" variables relevant to the search term
    """
    
    url = 'https://api.nal.usda.gov/ndb/search/'
    # can set the website url parameters using dictionary and requests.get will build them out
    p = {'format':'json', 'api_key':key, 'q':search_term, 'output':'basic', 'sort':'n'}
    r = requests.get(url, params = p)
    #print("\nURL: "+ r.url + "\n")
    
    #select from r.json() list where we only get the element, 'item' : [stuff we want]   
    data = r.json()
    item = data['list']['item']

    return(item)
    
    
ndb_search('quail eggs',my_api_key)

[{'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'BUDDHA, QUAIL EGGS IN BRINE, UPC: 761934535098',
  'ndbno': '45099560',
  'offset': 0},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'CHAOKOH, QUAIL EGG IN BRINE, UPC: 044738074186',
  'ndbno': '45094707',
  'offset': 1},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': "D'ARTAGNAN, QUAIL EGGS, UPC: 736622102630",
  'ndbno': '45178254',
  'offset': 2},
 {'ds': 'SR',
  'group': 'Dairy and Egg Products',
  'name': 'Egg, quail, whole, fresh, raw',
  'ndbno': '01140',
  'offset': 3},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'GRAN SABANA, QUAIL EGGS, UPC: 819140010103',
  'ndbno': '45169279',
  'offset': 4},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'L&W, QUAIL EGGS, UPC: 024072000256',
  'ndbno': '45094890',
  'offset': 5}]

__Exercise 1.2__ (20 pts)

Use your search function to get NDB numbers for the foods in the `fresh.csv` file. It's okay if you don't get an NDB number for every food, but try to come up with a strategy that gets most of them. Discuss your strategy in a short paragraph.

Hints:

* The foods are all raw and unbranded.
* You can test search terms with the [online search page](https://ndb.nal.usda.gov/ndb/search/list).
* You can convert the output of `ndb_search()` to a data frame with `pd.DataFrame()`.
* The string methods for [Python](https://docs.python.org/2/library/stdtypes.html#string-methods) and [Pandas](http://pandas.pydata.org/pandas-docs/stable/text.html#method-summary) are useful here. It's okay if you use _simple_ regular expressions in the Pandas methods, although this exercise can be solved without them.
* You can merge data frames that have a column in common with `pd.merge()`.

In [5]:
import pandas as pd
import numpy as np
import os
import sys
!{sys.executable} -m pip install -e git+https://github.com/pwdyson/inflect.py#egg=inflect
import inflect
p = inflect.engine()

# make sure "fresh.csv" file is in same Jupyter directory as this notebook
try:
    ind = os.listdir().index('fresh.csv')
    file = os.listdir()[ind]
except ValueError:
    print('\nERROR:\n    Make sure "fresh.csv" is in working directory.\n')

fresh = pd.read_csv(file)
fresh = pd.DataFrame(fresh)
food = fresh['food']

# accounting for underscores in food names for search function
food_list = []
for i in food:
    if i.find("_") == -1:
        food_list.append(i)
    else:
        i = i.split('_')
        word = i[0] + " " + i[1]
        food_list.append(word)

def name_parser(api_list):
    """
    Input: 
        list format from the API with variables "ds", "name", "ndbno", etc.
    Returns:
        Formatted decomposition of the "name" variable in the list which comes with many identifying sentences.
    """
    name = api_list['name'].lower()
    decomp = name.replace(",", "").replace("(","").replace(")","").replace("-"," ").split(" ")
    raw1 = name.find("upc") == -1
    raw2 = api_list['ds'] == 'SR'
    if raw1 & raw2:
        return(decomp)
    
        
# after analysis I realized all I needed was the column "ds" equal to "SR" for fresh fruit
names = []
for food in food_list:
    try: # need this for "kiwi" search since there are no occurances of fresh raw kiwi
        x = ndb_search(food + ", raw",my_api_key)
        for i in x:
            n = name_parser(i)
            if n != None:
                names.append([n,i['ndbno']])
                #print(n, str(i['ndbno']))
    except KeyError:
        continue
df = pd.DataFrame()
lis = []
#Utilizing "inflect" package for checking singular and plural fruit name cases
for i in names:
    for j in food_list:
        # checks is our csv file food names in API list "name"
        if j in i[0]:
            #print(j + ": "+ str(i[1]))
            lis.append(j+","+i[1])
        # if not it checks the plural case of j
        elif ((j not in i[0]) & (p.singular_noun(j) == False)):
            j2 = p.plural_noun(j)
            if j2 in i[0]:
                #print(j2 + ": "+ str(i[1]))
                lis.append(j2+","+i[1])
        # if not it checks the singular case of j
        elif ((j not in i[0]) & (p.plural_noun(j)[-2:] == "ss")):
            j3 = p.singular_noun(j)
            if j3 in i[0]:
                lis.append(j3+","+i[1])
df = []                
for i in lis:
    sp = i.split(",")
    food = p.plural_noun(sp[0])
    if food[-2:] == "ss":
        food = food[:-1]
    ndbno = sp[1]
    df.append([food,ndbno])

dataframe = pd.DataFrame(df, columns=['Food',"NDB Number"])
dataframe = dataframe.sort_values(by=['Food'])
dataframe = dataframe.set_index('Food')
dataframe

Obtaining inflect from git+https://github.com/pwdyson/inflect.py#egg=inflect
  Updating c:\cygwin64\home\pc_owner\sta141b\jupyter_notebooks\davis\src\inflect clone
  Found existing installation: inflect 0.2.5
    Uninstalling inflect-0.2.5:
      Successfully uninstalled inflect-0.2.5
  Running setup.py develop for inflect


NameError: name 'inflect' is not defined

__Exercise 1.3__ (20 pts)

Read the [food reports V2 documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-FOOD-REPORTV2.md), then write a function called `ndb_report()` that requests a _basic_ food report. The function should accept the NDB number as an argument and return the list of nutrients for the food.

Note that the report url is: `https://api.nal.usda.gov/ndb/V2/reports`

For example, for `"09279"` (raw plums) the first element of the returned list should be:

```python
{u'group': u'Proximates',
 u'measures': [{u'eqv': 165.0,
   u'eunit': u'g',
   u'label': u'cup, sliced',
   u'qty': 1.0,
   u'value': u'143.93'},
  {u'eqv': 66.0,
   u'eunit': u'g',
   u'label': u'fruit (2-1/8" dia)',
   u'qty': 1.0,
   u'value': u'57.57'},
  {u'eqv': 151.0,
   u'eunit': u'g',
   u'label': u'NLEA serving',
   u'qty': 1.0,
   u'value': u'131.72'}],
 u'name': u'Water',
 u'nutrient_id': u'255',
 u'unit': u'g',
 u'value': u'87.23'}
```

Be sure to document and test your function.

__Exercise 1.4__ (20 pts) Which foods provide the best combination of price, yield, and nutrition? You can use kilocalories as a measure of "nutrition" here, but more a detailed analysis is better. Use plots to support your analysis.